<a href="https://colab.research.google.com/github/Alex-Gnn2813/Projects-Profession-AI-Data-Analytics-course/blob/main/Gestione_NegozzioVegano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Project: Vegan product store software**


##**Section 1:** *the functions*

###***Useful functions*** in managing the program :

####*verify* & *new_file*

In [ ]:
"""
verify(): Verifies that the file exists. If the file doesn't exist, new_file() creates it. It will be called twice in the program.
"""

def verify(address):
  try:
    file_data=open (address,"r",newline='',encoding="utf-8")
    file_data.close()
  except FileNotFoundError:
    ""
    new_file(address)


In [ ]:
"""
new_file(): create a new file
"""

def new_file(address):
  file_data=open (address,"w",newline='',encoding="utf-8")
  csv_write=csv.DictWriter(file_data,fieldnames=["PRODOTTO","QUANTITA'","PREZZO","PREZZO_ACQUISTO"])
  csv_write.writeheader()
  file_data.close()


####*error_handling*

In [ ]:
"""
error_handling(): shows an error message if the given data type isn't coherent and allows you to re-enter it.
"""

def error_handling(type_d,message,comma):
  in_mess=None
  while in_mess==None:
    try:
      in_mess=type_d(input(message))
      assert in_mess>=0
    except Exception:
      in_mess=None
      if comma=="y":
        add_m=" ne numeri decimali "
      else:
        add_m=" "
      print(f"In questo campo NON sono accetati ne numeri negativi{add_m}ne lettere.\nRiprova.")
  return in_mess


###***Functions features*** of the program :

####*help*

In [ ]:
"""
help(): Returns a menu with available commands. It will be called twice in the program.
"""

def help():
  return ("I comandi disponibili sono i seguenti:\n"+
         "  • aggiungi: aggiungi un prodotto al magazzino\n"+
         "  • elenca: elenca i prodotti in magazzino\n"+
         "  • vendita: reggistra una vendita effetuata\n"+
         "  • profitti: mostra i profitti totali\n"+
         "  • aiuto: mostra i possibili comandi\n"+
         "  • chiudi: esci dal programma"
         )


####*add_product*

In [ ]:
"""
add_product(): checks whether the product is in stock or not, then adds or modifies, the quantity, of the inserted product.
"""

def add_product(pandas_df,name_a,quantity_a,address):

    prove=pandas_df[pandas_df["PRODOTTO"]==name_a].index.tolist()

    if len(prove):
      prove=int(prove[0])
      pandas_df.at[prove,"QUANTITA'"]= int(pandas_df.at[prove,"QUANTITA'"])+quantity_a
      print("AGGIUNTO: %d X %s" % (pandas_df.at[prove,"QUANTITA'"], name_a))

    else:
      p_price=error_handling(float,"Prezzo di acquisto: ","n")
      price=error_handling(float,"Prezzo di vendita: ","n")

      row=[name_a,quantity_a,price,p_price]
      new_row=pd.DataFrame(columns=pandas_df.columns,data=[row])
      pandas_df=pd.concat([pandas_df,new_row])
      print(f"AGGIUNTO: {quantity_a} X {name_a}")

    pandas_df.to_csv(address,index=False)


####*itemize*

In [ ]:
"""
itemize(): reads three columns of the csv file and shows it on the screen.
"""

def itemize(address):
  doc=open(address,"r")
  csv_read=csv.DictReader(doc)
  print("%s %s %s"% tuple(csv_read.fieldnames[:3]))
  for row in csv_read:
    print("%s %s €%.2f" %(row["PRODOTTO"],row["QUANTITA'"],float(row["PREZZO"])))
  doc.close()


####*add_selling*

In [ ]:
"""
add_selling(): provides a choice, whether to continue the cycle or interrupt it;
when the cycle is interrupted, it only shows the sales made during the current call.

"""

def add_selling(pandas_df,address1,address2):
  sales_l=[]
  request=None
  while request!="no":
    try:
      values=selling(pandas_df,address1,address2)
      sales_l.append(values)
      i=0
      while i==0:
        try:
          request=input("Aggiungere un altro prodotto? (si/no): ")
          request=request.lower()
          assert request in ["si","no"]
          i+=1
        except AssertionError:
          print("Risposta non valida.\nRiprova.")

      if request=="si":
          continue
      elif request=="no":
          print("VENDITA REGISTRATA")
          tot=[]
          for i in range(len(sales_l)):
            print(f"-{sales_l[i][1]} X {sales_l[i][0]}: €{sales_l[i][2]:.2f}")
            tot.append(sales_l[i][1]*sales_l[i][2])
          print(f"Totale: €{sum(tot):.2f}")
    except AssertionError:
      print("Attenzione!\nLa quantità del prodotto che vuoi vendere è maggiore della quantità presente in magazzino.\nRiprova.")
    except IndexError :
      print("Il prodotto inserito non è in magazzino.\nRiprova.")


####*selling*

In [ ]:
"""
selling(): change the quantity of the product in stock;
adds the sales to a file (which will be used in profit());
"""

def selling(pandas_df,address1,address2):
   name=input("Nome del prodotto: ")
   quantity=error_handling(int,"Quantità: ","y")

   prove=pandas_df[pandas_df["PRODOTTO"]==name].index.tolist()
   prove=int(prove[0])
   assert pandas_df.at[prove,"QUANTITA'"]>=quantity
   pandas_df.at[prove,"QUANTITA'"]= int(pandas_df.at[prove,"QUANTITA'"])-quantity
   data_p=pandas_df.copy()
   verify(address1)
   df_2=pd.read_csv(address1)
   values=[name,quantity,float(data_p.at[prove,"PREZZO"]),float(data_p.at[prove,"PREZZO_ACQUISTO"])]
   new_row=pd.DataFrame(columns=pandas_df.columns,data=[values])
   df_2=pd.concat([df_2,new_row])
   df_2.to_csv(address1,index=False)

   pandas_df["QUANTITA'"] = pandas_df["QUANTITA'"].replace(0, np.nan)
   pandas_df=pandas_df.dropna()
   pandas_df.to_csv(address2,index=False)

   return values


####*profit*

In [ ]:
"""
profit(): reads a second csv file, calculates and shows the gross and net profit.
All sales (including past ones) are included in this calculation.
"""

def profit(address):
   csv_2=open(address,"r")
   reader=csv.reader(csv_2)
   sales_l=list(reader)
   tot_l=[]
   tot_n=[]
   for i in range(1,len(sales_l)):
     tot_l.append(float(sales_l[i][1])*float(sales_l[i][2]))
   gross=(sum(tot_l))
   for j in range(1,len(sales_l)):
     tot_n.append(float(sales_l[j][1])*float(sales_l[j][3]))
   net=(gross-(sum(tot_n)))
   csv_2.close()
   print(f"Profitto: lordo=€{gross:.2f} netto=€{net:.2f}")


##**Section 2:** *the main part*

In [ ]:
"""
This is the main block of the program, where all functions find their use.
"""

import csv
import pandas as pd
import numpy as np


dnv_csv="/content/drive/MyDrive/Dati_NegozioVegano.csv"
v_csv="/content/drive/MyDrive/vendite.csv"
verify(dnv_csv)
control=None
while control!="chiudi":
  df=pd.read_csv(dnv_csv)
  file_data=open (dnv_csv,"w")

  control=input("Inserisci un comando: ")
  control=control.lower()
  if control=="aggiungi":
    name=input("Nome del prodotto: ")
    quantity=error_handling(int,"Quantità: ","y")
    add_product(df,name,quantity,dnv_csv)

  elif control=="elenca":
    df.to_csv(dnv_csv,index=False)
    itemize(dnv_csv)

  elif control=="vendita":
    add_selling(df,v_csv,dnv_csv)

  elif control=="profitti":
    df.to_csv(dnv_csv,index=False)
    profit(v_csv)

  elif control=="aiuto":
    df.to_csv(dnv_csv,index=False)
    print(help())

  elif control=="chiudi":
    df.to_csv(dnv_csv,index=False)
    file_data.close()
    print("Bye bye")

  else:
    df.to_csv(dnv_csv,index=False)
    print(f"Comando non valido\n{help()}")
